# TP: Introduction au deep learning avec `PyTorch`

`PyTorch` est un framework d'apprentissage automatique open source développé principalement par Facebook's AI Research Lab. Il est basé sur le langage de programmation Python et est connu pour sa flexibilité et sa facilité d'utilisation. `PyTorch` est largement utilisé pour la création et l'entraînement de réseaux de neurones, en particulier dans les domaines de la vision par ordinateur et du traitement du langage naturel.

Le coeur de `PyTorch` est écrit en `C++` pour des raisons de performances. Il utilise des bibliothèques mathématiques et d'algèbre linéaire bien établies, telles que `BLAS` (Basic Linear Algebra Subprograms) et `LAPACK` (Linear Algebra Package), pour accélérer les calculs numériques.

`PyTorch` fournit une interface Python conviviale pour interagir avec les fonctionnalités de bas niveau. Il utilise une approche orientée objet, ce qui signifie que les différentes fonctionnalités sont organisées en classes et objets. Les utilisateurs peuvent créer des instances de ces objets pour construire des modèles, définir des couches de réseau, définir des fonctions de perte, etc.

La bibliothèque `PyTorch` comprend également des modules pour le calcul automatique des gradients (**différentiation automatique**), qui est une fonctionnalité essentielle pour l'apprentissage automatique. Ces modules permettent de calculer automatiquement les dérivées des opérations effectuées sur les tenseurs.

## Jeu de données MNIST

MNIST (Modified National Institute of Standards and Technology) est une base de données largement utilisée dans le domaine de l'apprentissage automatique pour la reconnaissance de chiffres manuscrits. Il s'agit d'un ensemble de données contenant des images en niveaux de gris de chiffres manuscrits de 0 à 9.

La base de données MNIST est composée de deux parties principales :
- Le jeu d'apprentissage (training set) : Il comprend $60\,000$ exemples d'images de chiffres manuscrits, chacune étant une image de 28x28 pixels. Ces images sont associées à des étiquettes (labels) qui indiquent le chiffre correspondant (de 0 à 9).
- Le jeu de test (test set) : Il comprend $10\,000$ exemples supplémentaires d'images de chiffres manuscrits, également de taille 28x28 pixels. Les images de test sont utilisées pour évaluer les performances des modèles d'apprentissage automatique entraînés sur le jeu d'apprentissage.

La base de données MNIST est souvent utilisée comme un point de départ pour les tâches de classification d'images et pour l'évaluation des algorithmes d'apprentissage automatique. De nombreux chercheurs et développeurs utilisent MNIST pour tester de nouvelles architectures de réseaux de neurones et de nouvelles techniques d'apprentissage.

En raison de sa simplicité et de sa taille relativement petite, MNIST est devenu un jeu de données standard pour la communauté de l'apprentissage automatique. Il est souvent utilisé pour illustrer des concepts tels que la préparation des données, la classification, la rétropropagation, l'optimisation et la régularisation.

In [ ]:
import numpy as np
import torch

### Chargement des données

Le module `torchvision.datasets` est une composante du module `PyTorch` spécialisée dans le chargement et la gestion de jeux de données populaires: MNIST, CIFAR-10, ImageNet, etc.

La classe principale dans `torchvision.datasets` est Dataset, qui définit l'interface de base pour les jeux de données. Les classes spécifiques à chaque jeu de données héritent de cette classe et la personnalisent en fonction des caractéristiques spécifiques du jeu de données. Par exemple, `MNIST` est une sous-classe de `Dataset` et voici un appel standard: 
```python
from torchvision import datasets
mnist_train = datasets.MNIST('data', train=True, download=True)
```
Une fois l'instance du jeu de données créée, vous pouvez accéder aux exemples de données et à leurs étiquettes à l'aide d'indexing (opérateur `[]`).

- Créer un objet `mnist_train` de la classe `datasets.MNIST` (chargement des données).
- Déterminer le type des éléments puis afficher quelques images en indiquant les labels associés.
- Faire de même pour le dataset de test: `mnist_test`.

### Solution

In [ ]:
from torchvision import datasets
mnist_train = datasets.MNIST('data', train=True, download=True)

In [ ]:
image, label = mnist_train[0]

In [ ]:
print(image)
print(label)

`PIL.Image.Image` fait référence à la classe principale de la bibliothèque Python `PIL` (Python Imaging Library) ou de sa version améliorée, appelée `Pillow`. Cette classe représente une image dans PIL/Pillow. 
On peut visualiser l'image via la fonction `imshow` de `matplotlib.pyplot`.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=6, nrows=3)
for ax in axs.flatten():
    idx = torch.randint(len(mnist_train), size=(1,)).item()
    image, label = mnist_train[idx]
    ax.imshow(image, cmap="gray")
    ax.set_axis_off()
    ax.set_title(label)

### Transformation des images 

- A partir de `mnist_train` et des champs `data` et `targets`, créer un tensor `x_train` de shape 1x28x28 dont les données sont normalisées dans $[0,1]$ (0 pour un pixel blanc et 1 pour un pixel noir, il faut donc diviser par 255) et un tensor `y_train`. La première dimension (qui semble inutile) est nécessaire car les images en `PyTorch` sont manipulables sous le format `[channels, height, width]`, une image en couleur a 3 canaux R-G-B.
- Faire de même un `x_test` et `y_test` à partir de `mnist_test`.

**Remarque**:
Le module `transforms` du package `torchvision` fournit une collection de transformations couramment utilisées pour prétraiter les images de manière à les adapter aux besoins spécifiques de l'apprentissage automatique, tels que le redimensionnement, le recadrage, la normalisation, etc. 
On peut utiliser la fonction `ToTensor` pour convertir une image du dataset `mnist_train` en un tensor `PyTorch` et faire un appel de `datasets.MNIST` avec l'option `transform` pour obtenir directement les données sous la forme de tensors. 

Dans la suite on travaille avec ces données `x_train`, `y_train`, `x_test` et `y_test`.

### Solution

In [ ]:
mnist_train.data.shape

In [ ]:
x_train = mnist_train.data.reshape((-1,1,28,28)) / 255
print("shape: ", x_train.shape)
print("valeur min: ", x_train.min())
print("valeur max: ", x_train.max())
y_train = mnist_train.targets 

In [ ]:
y_train

In [ ]:
mnist_test = datasets.MNIST('data', train=False, download=True)
x_test = mnist_test.data.reshape((-1,1,28,28)) / 255
y_test = mnist_test.targets
print(y_test.shape)

### Question: Exploration du dataset  

- Parcourir les deux datasets pour déterminer la proportion des différents labels présents. Avant toute modélisation il est important de voir si les données sont homogènes: c'est à dire correctement réparties dans le train set et dans le test set. 

### Solution

In [ ]:
print(torch.bincount(y_train, minlength=10))
print(torch.bincount(y_test, minlength=10))

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_theme()

# Number of classes
num_labels = 10

y_counts = pd.DataFrame({
    'data': np.array(['train'] * num_labels + ['test'] * num_labels),
    'label': np.tile(np.arange(num_labels), 2),
    'prop': np.hstack([np.bincount(y_train) / len(y_train), 
                       np.bincount(y_test) / len(y_test)])
})

fig, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x='label', y='prop', hue='data', data=y_counts, ax=ax)

## Un premier modèle: régression softmax

Dans cette première partie, nous n'allons pas utiliser le fait que nos données $x$ sont des images. En effet, chaque image $x$ est une matrice $p\times p = 28\times 28$ ($x=(x_{ij})$) avec une structure de voisinage spécifique. Mais ici, nous traitons chaque image $x$ comme un vecteur de taille $784$ ($x = (x_{j})$) et nous ignorons la structure de voisinage.

Nous souhaitons classer ces images ou, de manière équivalente, prédire le chiffre $k$ variant dans ${0, \ldots, 9}$ qu'elles représentent.
Un modèle simple permettant de le faire est la régression softmax (ou régression logistique multinomiale).


L'idée est de produire un score pour chaque image d'entrée $x$ en utilisant un modèle linéaire simple.
Pour cela, nous supposons que l'appartenance à une classe $k$ (correspondant au chiffre $k$) peut être exprimée par une somme pondérée des intensités de pixels, avec des poids $W_{k, 1}, \ldots, W_{k, 784}$ et un biais (ou intercept) $b_k$ qui capture une variabilité indépendante de l'entrée :
$$
    \text{score}_k(x) = \sum_{j=1}^{784} W_{k, j} x_j + b_k,
$$
Ces scores sont parfois appelés "logits" dans la communauté de l'apprentissage profond.
Ensuite, nous utilisons la fonction softmax pour convertir les scores en probabilités prédites $p_k=\mathbb{P}(y=k|x)$ :
$$
    \forall k =0,\ldots,9,\quad p_k(x) = \text{softmax}(\text{score}_k(x)) = \frac{\exp(\text{score}_k(x))}{\sum_{\ell =0}^{9}\exp(\text{score}_{\ell}(x))}.
$$

### Module `torch.nn`

`torch.nn` est un module de `PyTorch` qui fournit des outils et des classes pour construire et entraîner des réseaux de neurones. Il fournit des blocs de construction pour définir les différentes couches et opérations nécessaires dans un réseau de neurones, ainsi que des fonctions d'activation, des fonctions de coût et d'autres fonctionnalités liées à l'apprentissage automatique.

Voici quelques éléments clés de `torch.nn`:

- Modules et couches: variété de modules et de couches pré-définis tels que `Linear`, `Conv2d`, `RNN`, `BatchNorm`, etc. Ces modules encapsulent des opérations spécifiques et sont utilisés pour construire des architectures de réseaux de neurones complexes.

- Fonctions d'activation: `ReLU`, `Sigmoid`, `Tanh`, etc., qui peuvent être appliquées aux sorties des couches pour introduire des non-linéarités dans le modèle.

- Fonctions de coût: `CrossEntropyLoss`, `MSELoss`, etc., qui sont utilisées pour évaluer la performance du modèle et guider l'apprentissage.

- Optimiseurs: `SGD`, `Adam`, `RMSprop`, etc., qui sont utilisés pour ajuster les poids du modèle pendant l'entraînement.

- Définition des réseaux de neurones personnalisés: on peut définir des réseaux de neurones personnalisés en créant des classes héritant de `torch.nn.Module`. Cela permet de définir des architectures complexes en combinant différentes couches et en définissant la logique de propagation avant (forward pass) à l'intérieur de la classe.

Pour créer rapidement et facilement des architectures de réseaux de neurones séquentiels, on peut utiliser le module `nn.Sequential`: il suffit de définir l'architecture d'un modèle en spécifiant simplement les couches dans l'ordre dans lequel elles doivent être appliquées. Cela permet de définit un modèle simple sans écrire de classe personnalisée héritant de `nn.Module`.

### Définition du modèle 

- En utilisant les modules `nn.Sequential`, `nn.Flatten` et `nn.Linear` définir un objet `model_linear` qui code la fonction $\text{score}_k: [0,1]^{784} \to \mathbf{R}^{10}$. En fait cette fonction sera appelée pour un ensemble d'images...

### Solution

In [ ]:
import torch.nn as nn

model_linear = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(28*28, 10), 
    # nn.Softmax(dim=1)  attention si on met cette couche on doit modifier le critère de loss dans la suite
)

In [ ]:
model_linear

In [ ]:
output = model_linear(x_train)
print(output.shape)
print(y_train.shape)

Un code similaire à la main serait le suivant 
```python
class MyNetwork(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(MyNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(dim_in, dim_out)
        #self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        # x = self.softmax(x)
        return x

model_linear = MyNetwork(28*28, 10)
```


### Accès aux paramètres du modèle

- Utiliser la méthode `parameters()` sur l'objet `model_linear` et sur les objets `model_linear[i]` (i=0,1) pour accéder aux paramètres des différentes couches.

- Sauver les paramètres (l'état) du modèle dans un dictionnaire `state_init`. On utilisera la méthode `state_dict()` et on pourra recharger cet état initial via la méthode `load_state_dict()`.

### Solution

In [ ]:
for p in model_linear.parameters():
    print(p)
    print(p.shape)

In [ ]:
for p in model_linear[0].parameters():
    print(p)

In [ ]:
for p in model_linear[1].parameters():
    print(p)

In [ ]:
import copy
state_init = copy.deepcopy(model_linear.state_dict())
state_init

### Fonction de perte et descente de gradient

Pour entraîner les paramètres du modèle (le biais $b_k$ et les poids $W_{k, j}$ où $k=0, \ldots, 9$ et $j=1, \ldots, 784$), la fonction de perte considérée (mesure de qualité) est le logarithme négatif de la vraisemblance défini par l'entropie croisée entre le score $\text{score}(x)=(\text{score}_0(x),\dots,\text{score}_9(x))$ et le vrai label $y=(y_0,\dots,y_9)$:
$$
    \ell\big( \text{score}(x), y \big) = - \sum_{k=0}^{9} y_{k} \log\big( \text{softmax}(\text{score}_k(x)) \big).
$$

Pour ce premier modèle, nous optimisons par rapport aux paramètres $(\boldsymbol{W}, \boldsymbol{b})$ la perte totale $F(\boldsymbol{W}, \boldsymbol{b})$ sur l'ensemble d'entraînement $(x^i,y^i){1\le i \le n_{\text{train}}}$ exprimée comme
$$
    F(\boldsymbol{W}, \boldsymbol{b}) = \sum_{i=1}^{n_{\text{train}}} \ell \big(\text{score}(x^i), y^i \big) = - \sum_{i=1}^{n_{\text{train}}} \sum_{k=0}^{9} y^i_{k} \log(p_k(x^i)).
$$

- Utiliser `nn.CrossEntropyLoss` pour la perte $\ell$ sur le dataset `mnist_train`. 

- Créer un objet `optimizer` de type `optim.SGD` qui est une classe du module `torch.optim` de PyTorch qui implémente l'algorithme de descente de gradient stochastique (SGD) pour l'optimisation des paramètres d'un modèle. Le paramètre `lr` (learning rate) sera fixé à 0.005. 

- Faire une boucle de 10 itérations qui parcourt 10 fois tout le dataset (`x_train`,`y_train`). À chaque itération, vous réinitialisez les gradients avec `optimizer.zero_grad()`, calculez les prédictions du modèle (sur tout le dataset), calculez la perte, effectuez la rétropropagation du gradient avec `loss.backward()`, puis mettez à jour les paramètres avec `optimizer.step()`. Cela applique les mises à jour des paramètres basées sur le gradient calculé automatiquement (différentiation automatique aussi connue sous le nom de rétropropagation du gradient).

- Une fois que le code fonctionne, passer à 200 epochs et tracer la loss, puis les accuracy (train et test) en fonction des epochs. 

### Solution

In [ ]:
criterion = nn.CrossEntropyLoss()
# Attention, si vous avez ajouté une couche `nn.Softmax` à votre modèle il faut utiliser un autre critère 
# (voir par exemple `nn.NLLLoss` negative log likelihood loss).

epochs = 10
for epoch in range(epochs):
    loss = criterion(model_linear(x_train), y_train)
    print(loss)

In [ ]:
from tqdm import tqdm
import torch.optim as optim

model_linear.load_state_dict(copy.deepcopy(state_init))
optimizer = optim.SGD(model_linear.parameters(), lr=0.01) # lr = learning rate
loss_train, accuracy_train, accuracy_test = [], [], [] 

n_train = len(y_train)
n_test = len(y_test)

epochs = 200
# for epoch in range(epochs): 
with tqdm(range(epochs), unit="epoch") as tepoch:
    for epoch in tepoch:
        optimizer.zero_grad() # a clean up step for PyTorch
        scores = model_linear(x_train)
        loss = criterion(scores, y_train)
        loss.backward()       # compute updates for each parameter
        optimizer.step()      # make the updates for each parameter
        
        loss_train.append(loss.item())
        correct = (scores.max(axis=1).indices.flatten() == y_train).sum().item()
        accuracy_train.append(correct / n_train)
        
        with torch.no_grad():
            correct = (model_linear(x_test).max(axis=1).indices.flatten() == y_test).sum().item()
            accuracy_test.append(correct / n_test)
        tepoch.set_postfix(loss=loss.item(), accuracy=accuracy_train[-1], accuracy_test=accuracy_test[-1])

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4), layout="tight")
ax1.plot(np.arange(epochs), loss_train, label="loss (full gradient)")
ax2.plot(np.arange(epochs), accuracy_train, label="accuracy train (full gradient)")
ax2.plot(np.arange(epochs), accuracy_test, label="accuracy test (full gradient)")
ax1.legend()
ax2.legend()
fig.suptitle("Full Gradient on linear model")
plt.show()

### Optimisation stochastique, mini-batch

Ecrire une fonction pour faire l'apprentissage d'un modèle en implémentant un algorithme de descente de gradient stochastique. Adapter le code précédent pour écrire une fonction `training` qui prend pour arguments:
- `model` un modèle qui renvoie les scores 
- `optimizer` (qu'on suppose bien initialisé avec les `model.parameters()`)
- `epochs` le nombre d'epochs, par défaut à 100
- `batch_size` par défaut à 512

A chaque itération de `epoch` la procédure est la suivante: 
- on mélange le dataset (shuffle)
- on parcourt le dataset par bloc de taille `batch_size`
- sur chaque bloc on calcule le gradient et on met à jour les paramètres du modèle

A la fin d'une itération `epoch` on recalcule `accuracy_train` et `accuracy_test` sur les données. L'**accuracy calculée est la proportion d'images bien prédites**.

**Remarque:** en `PyTorch` on peut utiliser un `DataLoader` pour faire le travail de fournir les données en paquets de taille `batch_size` mais pour des raisons de pédagogie on refait le code à la main à partir de `x_train` et `y_train`.

### Solution

In [ ]:
from tqdm import tqdm
def training(model, optimizer, epochs=100, batch_size=512): 
    criterion = nn.CrossEntropyLoss()
    loss_train, accuracy_train = [], [] 
    accuracy_test = []

    n_train = len(y_train)
    n_test = len(y_test)
    n_iters = n_train // batch_size
    with tqdm(range(epochs), unit="epoch") as tepoch:
        for epoch in tepoch:
            model.train()
            running_loss = 0
            correct = 0
            # on mélange à chaque epoch par une permutation aléatoire
            idx = torch.randperm(n_train)
            for k in range(n_iters):
                batch = idx[k*batch_size: min((k+1)*batch_size, n_train)]
                x_batch, y_batch = x_train[batch], y_train[batch] 
                
                optimizer.zero_grad()
                scores = model(x_batch)
                loss = criterion(scores, y_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            running_loss /= n_iters
            loss_train.append(running_loss)
            model.eval()
            with torch.no_grad():
                scores = model(x_train)
                correct = (scores.max(axis=1).indices.flatten() == y_train).sum().item()
                accuracy_train.append(correct / n_train)
                scores = model(x_test)
                correct = (scores.max(axis=1).indices.flatten() == y_test).sum().item()
                accuracy_test.append(correct / n_test)
            tepoch.set_postfix(loss=loss_train[-1], accuracy=accuracy_train[-1], accuracy_test=accuracy_test[-1])
    return { "loss": loss_train, "accuracy_train": accuracy_train, "accuracy_test": accuracy_test }

### Comparaison SGD et Adam

Utiliser la fonction précédente `training` pour comparer sur 200 epochs les 2 optimiseurs suivants:
- SGD Stochastic Gradient Descent classique avec learning rate de 0.005
- Adam (ADAptive Moment estimation) avec les paramètres par défaut

On tracera les loss en fonction des epochs ainsi que l'accurary score sur les données d'entrainement et sur les données d'apprentissage.

### Solution

In [ ]:
# on définit le dictionnaire result pour stocker tous les résultats d'entrainement
result = {}

In [ ]:
epochs = 200
result["softmax"] = { "epochs": np.arange(epochs) }

model_linear.load_state_dict(copy.deepcopy(state_init))
optimizer = optim.SGD(model_linear.parameters(), lr=0.005)
result["softmax"]["sgd"] = training(model_linear, optimizer, batch_size=512, epochs=epochs)

In [ ]:
r = result["softmax"]
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4), layout="tight")
ax1.plot(r["epochs"], r["sgd"]["loss"], label="loss SGD")
ax2.plot(r["epochs"], r["sgd"]["accuracy_train"], label="accuracy train SGD")
ax2.plot(r["epochs"], r["sgd"]["accuracy_test"], label="accuracy test SGD")
ax1.legend()
ax2.legend()
fig.suptitle("Softmax regression")
plt.show()

In [ ]:
model_linear.load_state_dict(copy.deepcopy(state_init))
optimizer = optim.Adam(model_linear.parameters())
result["softmax"]["adam"] = training(model_linear, optimizer, batch_size=512, epochs=epochs)

In [ ]:
# pour simplifier les codes ultérieurs on définit une fonction... 
def plot_SGD_Adam(r, name): 
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4), layout="tight")
    ax1.plot(r["epochs"], r["sgd"]["loss"], label="loss SGD")
    ax1.plot(r["epochs"], r["adam"]["loss"], color="C2", label="loss Adam")
    ax2.plot(r["epochs"], r["sgd"]["accuracy_train"], label="accuracy train SGD")
    ax2.plot(r["epochs"], r["sgd"]["accuracy_test"], label="accuracy test SGD")
    ax2.plot(r["epochs"], r["adam"]["accuracy_train"], color="C2", label="accuracy train Adam")
    ax2.plot(r["epochs"], r["adam"]["accuracy_test"], color="C3", label="accuracy test Adam")
    ax1.legend()
    ax2.legend()
    fig.suptitle(name)
    plt.show()

In [ ]:
plot_SGD_Adam(result["softmax"], "Softmax regression")

### Changement de backend (si possible)

En informatique, le terme "backend" fait référence à la partie d'un logiciel ou d'un framework qui est responsable de l'exécution des opérations de bas niveau, telles que les calculs mathématiques, l'optimisation et l'interaction avec le matériel. En `PyTorch`, un backend est la composante responsable de l'exécution des opérations sur les tenseurs et de la gestion des calculs.
`PyTorch` propose plusieurs backends pour exécuter les opérations de bas niveau. Les principaux backends de `PyTorch` sont les suivants :

- CPU Backend: `"cpu"` les calculs sont effectués sur le processeur central (CPU) de l'ordinateur. Il offre une exécution efficace sur les CPUs modernes et est capable de tirer parti des optimisations spécifiques du matériel.

- CUDA Backend: `"cuda"` CUDA est une plateforme de calcul parallèle développée par NVIDIA. `PyTorch` utilise le backend CUDA pour exécuter des calculs sur les GPU NVIDIA. L'utilisation de GPU permet d'accélérer considérablement les calculs en parallélisant les opérations sur des milliers de coeurs de traitement. Cela rend `PyTorch` particulièrement efficace pour l'entraînement et l'inférence de modèles de réseaux de neurones profonds.

- Autres backends: `"mps"` pour le GPU intégré des puces M1/M2 chez Apple.

### Solution

In [ ]:
# sur mon macbook pro M2 
mps_device = torch.device("mps")
x_train = x_train.to(mps_device)
y_train = y_train.to(mps_device)
x_test = x_test.to(mps_device)
y_test = y_test.to(mps_device)

epochs = 200
model_linear.load_state_dict(copy.deepcopy(state_init))
model_linear.to(mps_device)
optimizer = optim.Adam(model_linear.parameters())
training(model_linear, optimizer, batch_size=512, epochs=epochs);

In [ ]:
# on revient sur le cpu pour la suite... 
cpu_device = torch.device("cpu")
x_train = x_train.to(cpu_device)
y_train = y_train.to(cpu_device)
x_test = x_test.to(cpu_device)
y_test = y_test.to(cpu_device)

## Feed-forward neural network (FFNN)

On ajoute une couche cachée (hidden layer) entre les entrées et la couche de sortie linéaire de 10 neurones. Cette couche cachée est une couche linéaire avec une fonction non-linéaire appliquée point par point. La fonction non-linéaire classique que l'on utilise ici est la fonction **Rectified Linear Unit**: $\text{ReLU}(x) = \max(x, 0)$. 

### Définition du modèle
- Définir un `model_ffnn` avec une seule couche cachée de 128 neurones. Cette couche a pour fonction d'activation un ReLU.
- Combien de paramètres possède cette fonction ? C'est la dimension dans lequel on doit résoudre le problème d'optimisation! 
- Reprendre les questions précédentes avec ce modèle.

### Solution

In [ ]:
hidden_layer = 128

model_ffnn = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, hidden_layer), 
    nn.Dropout(p=0.1), 
    nn.ReLU(),
    nn.Linear(hidden_layer, 10)
)
print(model_ffnn)

import copy
state_init_ffnn = copy.deepcopy(model_ffnn.state_dict())
state_init_ffnn

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(model_ffnn)

In [ ]:
from torchinfo import summary

In [ ]:
summary(model_ffnn)

In [ ]:
epochs = 200
result["ffnn"] = { "epochs": np.arange(epochs) } 

model_ffnn.load_state_dict(copy.deepcopy(state_init_ffnn))
optimizer = optim.SGD(model_ffnn.parameters(), lr=0.005)
result["ffnn"]["sgd"] = training(model_ffnn, optimizer, batch_size=512, epochs=epochs)

In [ ]:
model_ffnn.load_state_dict(copy.deepcopy(state_init_ffnn))
optimizer = optim.Adam(model_ffnn.parameters())
result["ffnn"]["adam"] = training(model_ffnn, optimizer, batch_size=512, epochs=epochs)

In [ ]:
plot_SGD_Adam(result["ffnn"], "Feed-Forward Neural Network (1 hidden layer)")

### Visualisation des exemples mal classés

- Combien d'exemples sont mal classés par ce modèle après apprentissage et optimisation via Adam ? 
- Visualiser quelques exemples mal classés par ce modèle.

### Solution

In [ ]:
scores_test = model_ffnn(x_test)
preds = scores_test.max(axis=1).indices.flatten()
correct = preds == y_test
num_uncorrect = torch.logical_not(correct).sum().item()
print("nombre d'exemples mal classés:", num_uncorrect)

In [ ]:
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
transform = ToPILImage()

fig, axs = plt.subplots(ncols=6, nrows=3)
for ax in axs.flatten():
    idx = torch.randint(num_uncorrect, size=(1,)).item()
    image = transform(x_test[torch.logical_not(correct)][idx])
    pred = preds[torch.logical_not(correct)][idx].numpy()
    label = y_test[torch.logical_not(correct)][idx].numpy()
    ax.imshow(image, cmap="gray")
    ax.set_axis_off()
    ax.set_title(np.array2string(pred) + "  -  " + np.array2string(label))

## Convolutional Neural Network (CNN)

Un réseau neuronal convolutif (Convolutional Neural Network ou CNN en anglais) est une architecture spécifique de réseau neuronal profond (deep learning) conçue principalement pour le traitement des données structurées en grille, telles que des images ou des séquences temporelles. Les CNN sont particulièrement performants dans les tâches de vision par ordinateur, comme la classification d'images, la détection d'objets et la segmentation sémantique.

La principale caractéristique d'un CNN réside dans son utilisation de couches de convolution. Les couches de convolution effectuent des opérations de convolution sur les entrées, en utilisant des filtres ou des noyaux appris pour extraire des caractéristiques spécifiques des données: les bords, les textures ou les motifs présents dans les données.

Une couche de convolution possède plusieurs hyperparamètres: 

En plus des couches de convolution, les CNN comprennent également d'autres types de couches:

- Couches de pooling : Les opérations de pooling, comme le max pooling, aident à préserver les caractéristiques les plus importantes tout en réduisant la quantité de calculs nécessaires et le nombre de paramètres.

- Couches de normalisation : Elles normalisent les activations des neurones pour améliorer la stabilité et accélérer l'apprentissage.

- Couches entièrement connectées : Elles sont situées à la fin du réseau et réalisent une classification ou une régression sur les caractéristiques extraites par les couches précédentes. Ces couches sont similaires aux couches d'un réseau neuronal traditionnel et utilisent généralement des fonctions d'activation non linéaires telles que ReLU (Rectified Linear Unit).

![](img/cnn_view.png)

Voici par exemple des architectures "historiques", obtenues sur la page [wikipedia de LeNet](https://en.wikipedia.org/wiki/LeNet).
![](img/cnn_examples.svg)

Ici on propose de tester une architecture basée sur: 
- `nn.Conv2d`: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
- `nn.MaxPool2d`: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
- `nn.Linear`

### Définition du modèle 

Définir le réseau CNN suivant:
- Convolution avec 8 kernels de taille 5 (c'est à dire un carré 5x5), stride et padding à 0
- ReLU
- Max-Pooling 2x2
- Convolution avec 16 kernels de taille 5, stride et padding à 0
- ReLU
- Max-Pooling 2x2
- Une simple couche linéaire de sortie

### Solution

In [ ]:
model_cnn = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2), # 12
    nn.Conv2d(8, 16, 5), 
    nn.ReLU(),
    nn.MaxPool2d(2), # 4
    nn.Flatten(),
    nn.Linear(16*4*4, 10), 
)

import copy
state_init_cnn = copy.deepcopy(model_cnn.state_dict())
#state_init_cnn

In [ ]:
count_parameters(model_cnn)

In [ ]:
# pour avoir des informations plus complètes sur l'architecture du réseau 
# on peut charger le module `torchinfo` (après installation)
from torchinfo import summary
summary(model_cnn, input_size=(512, 1, 28, 28))

In [ ]:
device = torch.device("cpu")
x_train = x_train.reshape((-1,1,28,28)).to(device)
x_test = x_test.reshape((-1,1,28,28)).to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
epochs = 50
result["cnn"] = { "epochs": np.arange(epochs) } 

model_cnn.load_state_dict(copy.deepcopy(state_init_cnn))
model_cnn.to(device)
optimizer = optim.SGD(model_cnn.parameters(), lr=0.005)
result["cnn"]["sgd"] = training(model_cnn, optimizer, batch_size=512, epochs=epochs)

In [ ]:
model_cnn.load_state_dict(copy.deepcopy(state_init_cnn))
model_cnn.to(device)
optimizer = optim.Adam(model_cnn.parameters())
result["cnn"]["adam"] = training(model_cnn, optimizer, batch_size=512, epochs=epochs)

In [ ]:
plot_SGD_Adam(result["cnn"], "Convolutional Neural Network")

In [ ]:
scores_test = model_cnn(x_test)
preds = scores_test.max(axis=1).indices.flatten()
correct = preds == y_test
num_uncorrect = torch.logical_not(correct).sum().item()
print("nombre d'exemples mal classés:", num_uncorrect)

## Changement de dataset: FashionMNIST

Refaire la comparaison FNN (fully-connected) vs CNN sur le jeu de données FashionMNIST.
```
datasets.FashionMNIST
```

In [ ]:
from torchvision import datasets
mnist_train = datasets.FashionMNIST('data', train=True, download=True)
mnist_test = datasets.FashionMNIST('data', train=False, download=True)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=6, nrows=3)
for ax in axs.flatten():
    idx = torch.randint(len(mnist_train), size=(1,)).item()
    image, label = mnist_train[idx]
    ax.imshow(image, cmap="gray")
    ax.set_axis_off()
    ax.set_title(label)

## Annexe

### Utilisation du `DataLoader`

Le `DataLoader` de `PyTorch` est une classe utilitaire qui facilite le chargement et la gestion des données d'entraînement et de test dans les modèles d'apprentissage automatique. Il offre plusieurs avantages et fonctionnalités qui simplifient le processus de préparation des données et d'itération sur les batchs (mini-lots) lors de l'entraînement des modèles. 

On reprend le jeu de données MNIST.
- Créer un `DataLoader` en spécifiant le jeu de données `mnist_train` comme argument, avec un `batch_size` de 64 pour regrouper les données en mini-lots de taille 64, et avec l'option `shuffle=True` pour mélanger aléatoirement les données à chaque epoch (après un passage complet sur l'ensemble du dataset).
- Faire une boucle sur cet objet `DataLoader` pour le manipuler.
- Que représente une itération complète sur cet objet, et combien d'images est dans le dernier lot ? 

### Solution

In [ ]:
from torchvision.transforms import ToTensor
mnist_train = datasets.MNIST('data', train=True, download=True, transform=ToTensor())

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)

In [ ]:
train_loader

In [ ]:
n = len(train_loader)

In [ ]:
for k, (x_batch, label) in enumerate(train_loader): 
    if k in (0, 1, n-2, n-1):
        print("lot: ", k)
        print("nombre d'images: ", len(x_batch))
        print("forme images: ", x_batch.shape)
        print("nombre de labels: ", len(label))